## Import Libraries

In [1]:
import requests
import json, os
from bs4 import BeautifulSoup as bs
import datetime

##Articles Scraper
- Title
- Date
- Content
- Summary
- ID

In [14]:
from requests.api import get
def get_id(url):
  '''Getting the ID of the news from its url'''
  return url.split('/')[-2]

def get_summary(text):
    summary = ''
    for line in text.split('\n'):
        if 'window.kmklabs.channel =' in line:
            target = line
            break
    temp=target.split('window.kmklabs.article = ')[1]
    temp=temp.split(';')[0]
    data = json.loads(temp)
    return data['shortDescription']

def collect_data(text):
  soup = bs(text)
  title = soup.find_all('h1', {'class': 'read-page--header--title'})[0].get_text()
  date = soup.find_all('time', {'class': 'read-page--header--author__datetime updated'})[0].get_text()
  contents = soup.find_all('div', {'class': 'article-content-body__item-content'})
  article = []
  for content in contents:
    article.append(content.get_text())
  summary = get_summary(text)
  return title, date, article, summary

def scrape_article(url):
  request = requests.get(url)
  url = request.url
  id = get_id(url)
  title, date, article, summary = collect_data(request.text)
  news_article = {
      'id': id,
      'title': title,
      'date': date,
      'article': article,
      'summary': summary
  }
  return news_article

def news_dataset_generator(url_list):
  dataset = []
  num_news = len(url_list)
  num_error = 0
  for i, url in enumerate(url_list):
    print('Scraping {}/{}'.format(i+1, num_news))
    try:
      news = scrape_article(url)
      dataset.append(news)
    except:
      num_error += 1
      print('Error scraping data {}/{}; ID {}; Error:{}'.format(i, num_news, get_id(url), num_error))
  return dataset


###Tes scraper

In [6]:

urls = ['https://www.liputan6.com/news/read/4934674/transjakarta-tutup-sementara-9-halte-mulai-15-april-2022', 'https://www.liputan6.com/news/read/4934691/panglima-andika-pastikan-tni-tidak-represif-saat-bantu-polri-jaga-demo-11-april-2022', 'https://www.liputan6.com/news/read/4951071/liputan6-update-laporan-mudik-dari-garut']
data = news_dataset_generator(urls)


Scraping 1/3
Scraping 2/3
Scraping 3/3


In [18]:
data

[{'article': ['Liputan6.com, Jakarta - PT Transjakarta berencana menutup sementara sembilan halte mulai 15 April 2022. Penutupan tersebut karena adanya revitalisasi di 11 halte bus Transjakarta.\n"Guna meningkatkan kelengkapan fasilitas dan kenyamanan pelanggan, Transjakarta akan melakukan revitalisasi di 11 halte. Mulai 15 April 2022, sembilan dari 11 halte akan dilakukan penutupan sementara," tulis Transjakarta dalam akun Twitter resminya @PT_Transjakarta di Jakarta, Sabtu (9/4/2022), seperti dikutip dari Antara.\n\n\nBaca Juga\n\nTransjakarta Perpanjang Jam Operasional Layanan hingga Pukul 22.00\nBus Transjakarta Sebabkan Kecelakaan Beruntun di Tol Jagorawi\nWagub DKI Jakarta soal Sudirman Said Jadi Komut Transjakarta: Punya Kemampuan\n\n\n\n\n\nSembilan halte Bus Transjakarta yang akan ditutup sementara mulai 15 April 2022 itu adalah:\n1. Halte Dukuh Atas 1\n2. Halte Tosari\n3. Halte Juanda\n4. Halte Cawang Cikoko\n5. Halte Bundaran HI\n6. Halte Sarinah\n7. Halte Kebon Pala\n8. Hal

##Get News URLs

In [ ]:
def generate_date_range(date_range=30):
  base = datetime.datetime.today()
  date_range = [base - datetime.timedelta(days=x) for x in range(date_range)]
  date_list = []
  for date in date_range:
    #print (date.strftime("%d-%m-%Y"))
    date_format = date.strftime("%Y/%m/%d")
    date_list.append(date_format)
  return date_list

def get_article_url(url):
  request = requests.get(url)
  soup = bs(request.text)
  url_list = []
  links = soup.find_all('a', {'class': 'articles--rows--item__title-link'})
  if links:
    for link in links:
      link = link.get('href')
      url_list.append(link)
    return url_list

def generate_news_index_url(channel, date_range=30):
  base_url = 'https://www.liputan6.com'
  date_list = generate_date_range(date_range)
  index_urls = []
  for date in date_list:
    url = '{}/{}/indeks/{}'.format(base_url, channel, date)
    for page in range(5):
      link = '{}?page={}'.format(url, page+1)
      index_urls.append(link)
  return index_urls

##Scrape Multiple Articles URLs

In [ ]:
# Lewatin ini dulu
'''
channels = ['news', 'bisnis']
url_list = []
for channel in channels:
  urls = generate_news_index_url(channel)
  for url in urls:
    url_list.append(url)

news_urls = []
for url in url_list:
  articles = get_article_url(url)
  for article in articles:
    news_urls.append(article)
'''

###Save URLs as JSON

In [9]:
def save_data(title, data):
  with open(title, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

def load_data(title):
  with open(title, encoding='utf-8') as f:
    return json.load(f)

In [ ]:
save_data('liputan6_news_links.json', news_urls)

##Start scraping from Liputan6

In [7]:
import requests
import json, os
from bs4 import BeautifulSoup as bs
import datetime

In [10]:
# Load data
news_urls = load_data('liputan6_news_links.json')

In [15]:
dataset = news_dataset_generator(news_urls)

Scraping 1/4453
Scraping 2/4453
Scraping 3/4453
Scraping 4/4453
Scraping 5/4453
Scraping 6/4453
Scraping 7/4453
Scraping 8/4453
Scraping 9/4453
Scraping 10/4453
Scraping 11/4453
Scraping 12/4453
Scraping 13/4453
Scraping 14/4453
Scraping 15/4453
Scraping 16/4453
Error scraping data 15/4453; ID 4958352; Error:1
Scraping 17/4453
Scraping 18/4453
Scraping 19/4453
Scraping 20/4453
Scraping 21/4453
Scraping 22/4453
Scraping 23/4453
Scraping 24/4453
Scraping 25/4453
Scraping 26/4453
Scraping 27/4453
Scraping 28/4453
Scraping 29/4453
Scraping 30/4453
Scraping 31/4453
Scraping 32/4453
Scraping 33/4453
Scraping 34/4453
Scraping 35/4453
Scraping 36/4453
Scraping 37/4453
Scraping 38/4453
Scraping 39/4453
Scraping 40/4453
Error scraping data 39/4453; ID 4958139; Error:2
Scraping 41/4453
Scraping 42/4453
Scraping 43/4453
Scraping 44/4453
Scraping 45/4453
Scraping 46/4453
Scraping 47/4453
Scraping 48/4453
Scraping 49/4453
Scraping 50/4453
Scraping 51/4453
Scraping 52/4453
Error scraping data 51/4453

In [16]:
save_data('liputan6_news_raw.json', dataset)